In [20]:
fn = '../data/raw/jstor_metadata_2025-11-28.jsonl.gz'

In [21]:
# !pip install orjsonl tqdm

In [22]:
from tqdm import tqdm
import json
import orjsonl

def iter_jsonl(fn):
    yield from orjsonl.stream(fn)

def search_jsonl(fn, issn = "00264423", total=12412004):
    for d in tqdm(iter_jsonl(fn), total=total):
        if d.get("c5_data_type") == "Journal":
            # issn_now = d.get("identifiers", {}).get("print_issn")
            # if issn == issn_now:
            # is_part_of = d.get("is_part_of", {})
            # if "Mind" in is_part_of:
            disciplines = d.get('discipline_names',[])
            if disciplines and 'Philosophy' in disciplines:
                yield d

In [4]:
from collections import Counter
all_journals = Counter(d['is_part_of'] for d in iter_jsonl(fn))

In [5]:
[k for k in all_journals if 'Philosophy Today' in str(k)]

[]

In [6]:
# next(search_jsonl(fn))

In [7]:
ld = list(search_jsonl(fn))

100%|██████████| 12412004/12412004 [00:35<00:00, 349910.78it/s]


In [23]:
df = pd.DataFrame(ld)
len(df)

448856

In [24]:
df.content_subtype.value_counts()

content_subtype
research-article      198283
book-review           172398
misc                   71863
news                    4021
frontmatter              961
backmatter               529
other                    268
editorial                229
discussion                61
review-essay              50
review-article            47
correction                39
books-received            32
introduction              22
announcement              21
index                     15
retraction                 4
book review                4
obituary                   2
letter                     2
backtmatter                1
article-commentary         1
addendum                   1
breif-report               1
product-review             1
Name: count, dtype: int64

In [25]:
jcounts = df.is_part_of.value_counts()

In [26]:
[x for x in jcounts.index if 'Speculative' in x]

['The Journal of Speculative Philosophy']

In [27]:
# df['date'] = pd.to_datetime(df['published_date'], errors='coerce')
df['year'] = df['published_date'].fillna('').apply(lambda x: x.split('-')[0] if isinstance(x, str) else '0').apply(int)
df['decade'] = df['year'] // 10 * 10

In [28]:
cdf=df.groupby('is_part_of').decade.value_counts()
cdf.columns=['count']
cdf = cdf.reset_index().pivot(columns='decade',index='is_part_of',values='count').fillna(0).applymap(int)
cdf['total'] = cdf.sum(axis=1)
cdf = cdf.sort_values('total',ascending=False)
cdf

/var/folders/py/jyzw5nyj1fnf0c_1czgsg2fc0000gn/T/ipykernel_23185/2871081181.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cdf = cdf.reset_index().pivot(columns='decade',index='is_part_of',values='count').fillna(0).applymap(int)


decade,1860,1870,1880,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020,total
is_part_of,,,,,,,,,,,,,,,,,,
Esprit,0,0,0,0,0,0,0,0,2162,3802,4403,5009,4361,3956,4643,4346,920,33602
Revue Philosophique de la France et de l'Étranger,0,651,1684,1917,2062,1486,986,848,764,1462,1040,2304,1974,2304,2063,1919,332,23796
Gregorianum,0,0,0,0,0,0,961,845,599,1619,2794,1517,2199,2095,1789,1597,0,16015
The Journal of Philosophy,0,0,0,0,0,0,1988,2810,2388,2348,1712,1409,1074,812,683,607,0,15831
The Journal of Symbolic Logic,0,0,0,0,0,0,0,502,1128,2328,2026,2071,1551,1467,962,845,82,12962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Selected Papers from the Annual Meeting (American Society of Christian Ethics),0,0,0,0,0,0,0,0,0,0,0,40,0,0,0,0,0,40
The Journal of Unified Science (Erkenntnis),0,0,0,0,0,0,0,20,12,0,0,0,0,0,0,0,0,32
Proceedings of the Meeting of the American Philosophical Association,0,0,0,0,12,12,8,0,0,0,0,0,0,0,0,0,0,32


In [29]:
# !pip install openpyxl
# cdf.to_excel('../data/philosophy_journals_article_counts.xlsx')

In [30]:
import pandas as pd
cdf2 = pd.read_excel('../data/philosophy_journals_article_counts-selected.xlsx')

In [31]:
jrnls = cdf2[cdf2.selected=="y"].journal.tolist()
jrnls

['The Journal of Philosophy',
 'The Philosophical Review',
 'The Review of Metaphysics',
 'Synthese',
 'Philosophy and Phenomenological Research',
 'Ethics',
 'International Journal of Ethics',
 'Erkenntnis (1975-)',
 'Dialectica',
 'Canadian Journal of Philosophy',
 'Proceedings of the Aristotelian Society',
 'Research in Phenomenology',
 'Erkenntnis (1930-1938)',
 'The Journal of Unified Science (Erkenntnis)']

In [32]:
# jrnl = ['The Philosophical Review', 'The Journal of Philosophy']
# jrnl = ['Research in Phenomenology','Philosophy and Phenomenological Research']
# jrnl = ['The Journal of Speculative Philosophy']
jdf = df[df.is_part_of.isin(jrnls)]
jdf.decade.value_counts().sort_index()

decade
1880      41
1890    1921
1900    2182
1910    2064
1920    3749
1930    5573
1940    5478
1950    6653
1960    6608
1970    7557
1980    8180
1990    8431
2000    8502
2010    9055
2020    2335
Name: count, dtype: int64

In [34]:
jdf.content_subtype.value_counts()

content_subtype
research-article    32783
book-review         27908
misc                14775
news                 2216
editorial             190
frontmatter           178
backmatter             78
other                  71
books-received         32
discussion             26
announcement           21
review-essay           20
correction             16
index                   9
review-article          4
introduction            1
addendum                1
Name: count, dtype: int64

In [40]:
finaldf = jdf[jdf.content_subtype=="research-article"]

In [41]:
item_ids = list(finaldf.item_id)
with open('../data/jstor_ids.txt','w') as of:
    of.write('\n'.join(item_ids)) 